# 🍈 MusangKing BALANCED Training v3
## Fixing Class Imbalance Issue

**Problem:** Model always predicts Musang King (73% of training data)

**Solution:**
- ⚖️ Undersample Musang King to balance classes
- 🎯 Use class weights in training
- 📊 Better feature normalization

In [ ]:
!pip install roboflow opencv-python-headless scikit-learn imbalanced-learn xgboost tqdm -q
print("✅ Dependencies installed!")

In [ ]:
# Load the features CSV from previous training (saves time!)
# Upload your features.csv from trained_models_v2/TRAINING_MODEL/
from google.colab import files
print("📤 Upload features.csv from trained_models_v2/TRAINING_MODEL/")
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
import xgboost as xgb
import joblib
import os

# Load features
df = pd.read_csv('features.csv')
print(f"Loaded {len(df)} samples")
print("\nOriginal Distribution:")
print(df['Variety'].value_counts())

## Step 1: BALANCE the Dataset

In [ ]:
# UNDERSAMPLE Musang King to match the average of other classes
min_samples = df['Variety'].value_counts().min()  # 858 (Black Thorn)
target_samples = int(min_samples * 1.5)  # ~1287 samples each

print(f"Target samples per class: ~{target_samples}")

# Balance by undersampling majority class
balanced_dfs = []
for variety in df['Variety'].unique():
    variety_df = df[df['Variety'] == variety]
    if len(variety_df) > target_samples:
        # Undersample
        variety_df = variety_df.sample(n=target_samples, random_state=42)
    balanced_dfs.append(variety_df)

df_balanced = pd.concat(balanced_dfs, ignore_index=True)
print("\n✅ Balanced Distribution:")
print(df_balanced['Variety'].value_counts())

## Step 2: Analyze Feature Differences

In [ ]:
# Check if features can actually distinguish varieties
print("=== Feature Means by Variety ===")
feature_cols = ['Compactness', 'Smoothness', 'Aspect_Ratio', 'Rectangularity', 'Mean_Red', 'Mean_Hue']
print(df_balanced.groupby('Variety')[feature_cols].mean())

print("\n=== Feature Standard Deviations ===")
print(df_balanced.groupby('Variety')[feature_cols].std())

## Step 3: Train with BALANCED Data

In [ ]:
print("="*60)
print("TRAINING VARIETY MODEL (BALANCED)")
print("="*60)

VARIETY_FEATURES = ['Compactness', 'Smoothness', 'Aspect_Ratio', 'Rectangularity', 'Mean_Red']

X = df_balanced[VARIETY_FEATURES]
y = df_balanced['Variety']

variety_encoder = LabelEncoder()
y_encoded = variety_encoder.fit_transform(y)
print(f"\nClasses: {variety_encoder.classes_}")

# Feature Scaling (IMPORTANT!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")
print(f"Train distribution: {pd.Series(y_train).value_counts().to_dict()}")

# Light SMOTE to even out any remaining imbalance
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
print(f"After SMOTE: {pd.Series(y_train_balanced).value_counts().to_dict()}")

In [ ]:
# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_balanced), y=y_train_balanced)
class_weight_dict = dict(zip(np.unique(y_train_balanced), class_weights))
print(f"Class weights: {class_weight_dict}")

# Build ensemble with class weights
print("\n🔧 Training Balanced Ensemble...")

svm = SVC(
    probability=True,
    kernel='rbf',
    C=10.0,
    gamma='scale',
    class_weight='balanced',
    random_state=42
)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)

xgb_clf = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=1,
    random_state=42,
    eval_metric='mlogloss'
)

gb = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)

# Equal voting weights
variety_model = VotingClassifier(
    estimators=[
        ('svm', svm),
        ('rf', rf),
        ('xgb', xgb_clf),
        ('gb', gb)
    ],
    voting='soft',
    weights=[1, 1, 1, 1]  # Equal weights
)

variety_model.fit(X_train_balanced, y_train_balanced)
print("✅ Training complete!")

In [ ]:
# Evaluate
y_pred = variety_model.predict(X_test)
y_pred_proba = variety_model.predict_proba(X_test)

print(f"\n🎯 Test Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=variety_encoder.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Check predictions per class
print("\n📊 Prediction Distribution on Test Set:")
pred_names = variety_encoder.inverse_transform(y_pred)
print(pd.Series(pred_names).value_counts())

# Confidence analysis
confidences = np.max(y_pred_proba, axis=1)
print(f"\n📈 Confidence: Mean={confidences.mean()*100:.1f}%, Min={confidences.min()*100:.1f}%, Max={confidences.max()*100:.1f}%")

## Step 4: Train Ripeness Model

In [ ]:
print("="*60)
print("TRAINING RIPENESS MODEL")
print("="*60)

RIPENESS_FEATURES = ['Mean_Hue', 'Compactness', 'Smoothness']

# Use Mean_Saturation if available, else create synthetic labels
if 'Mean_Saturation' in df_balanced.columns:
    df_balanced['Ripeness'] = df_balanced.apply(
        lambda row: 'defective' if row['Mean_Saturation'] < 50 or row['Compactness'] > 80 else 'mature', axis=1
    )
else:
    # Simple heuristic based on available features
    df_balanced['Ripeness'] = df_balanced.apply(
        lambda row: 'defective' if row['Compactness'] > 150 else 'mature', axis=1
    )

X_ripe = df_balanced[RIPENESS_FEATURES]
y_ripe = df_balanced['Ripeness']

ripeness_encoder = LabelEncoder()
y_ripe_encoded = ripeness_encoder.fit_transform(y_ripe)
print(f"Classes: {ripeness_encoder.classes_}")
print(f"Distribution: {pd.Series(y_ripe_encoded).value_counts().to_dict()}")

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_ripe, y_ripe_encoded, test_size=0.2, random_state=42, stratify=y_ripe_encoded
)

smote_r = SMOTE(random_state=42, k_neighbors=3)
X_train_r_smote, y_train_r_smote = smote_r.fit_resample(X_train_r, y_train_r)

ripeness_model = VotingClassifier(
    estimators=[
        ('svm', SVC(probability=True, class_weight='balanced', random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)),
        ('xgb', xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss'))
    ],
    voting='soft'
)

ripeness_model.fit(X_train_r_smote, y_train_r_smote)

y_pred_r = ripeness_model.predict(X_test_r)
print(f"\n🎯 Ripeness Accuracy: {accuracy_score(y_test_r, y_pred_r)*100:.2f}%")
print(classification_report(y_test_r, y_pred_r, target_names=ripeness_encoder.classes_))

## Step 5: Save & Download

In [ ]:
OUTPUT = "TRAINING_MODEL_BALANCED"
os.makedirs(OUTPUT, exist_ok=True)

joblib.dump(variety_model, f"{OUTPUT}/variety_model.pkl")
joblib.dump(variety_encoder, f"{OUTPUT}/variety_model_encoder.pkl")
joblib.dump(ripeness_model, f"{OUTPUT}/ripeness_model.pkl")
joblib.dump(ripeness_encoder, f"{OUTPUT}/ripeness_model_encoder.pkl")

print("✅ Models saved!")
!ls -la {OUTPUT}

In [ ]:
!zip -r balanced_models_v3.zip TRAINING_MODEL_BALANCED/

from google.colab import files
files.download('balanced_models_v3.zip')

print("\n🎉 BALANCED TRAINING COMPLETE!")
print("1. Extract ZIP")
print("2. Copy .pkl files to 'TRAINING MODEL' folder")
print("3. Restart Flask")